In [3]:
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import numpy as np
import cv2


# Prepare dataset

### Create directories

In [4]:
import os
import shutil


ROOT_DATA_DIR = 'dataset'
CLASSES = ["Burger", "Dimsum", "Ramen", "Sushi"]



shutil.rmtree(ROOT_DATA_DIR + '/train')
shutil.rmtree(ROOT_DATA_DIR + '/val')
shutil.rmtree(ROOT_DATA_DIR + '/test')

# # Creating Train / Val / Test folders (One time use)

for cls in CLASSES:
  os.makedirs(ROOT_DATA_DIR +'/train/' + cls)
  os.makedirs(ROOT_DATA_DIR +'/val/' + cls)
  os.makedirs(ROOT_DATA_DIR +'/test/' + cls)

### Load dataset

In [5]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# TRIM_MAX = 500

def loadDataset(current_class):
    src = "dataset/"+current_class # Folder to copy images from

    allFileNames = os.listdir(src)

    np.random.shuffle(allFileNames)
    train_fileNames, val_fileNames, test_fileNames = np.split(np.array(allFileNames),
                                                            [int(len(allFileNames)*(1 - (VAL_RATIO + TEST_RATIO))), 
                                                            int(len(allFileNames)*(1 - VAL_RATIO))])  

    train_fileNames = [src+'/'+ name for name in train_fileNames.tolist()]
    val_fileNames = [src+'/' + name for name in val_fileNames.tolist()]
    test_fileNames = [src+'/' + name for name in test_fileNames.tolist()]

    arr = range(len(train_fileNames))

    print(current_class, '\n\t Total images: ', len(allFileNames))
    print('\t Training: ', len(train_fileNames))
    print('\t Validation: ', len(val_fileNames))
    print('\t Testing: ', len(test_fileNames))

    # Copy-pasting images
    for name in train_fileNames:
        shutil.copy(name, ROOT_DATA_DIR + "/train/" + current_class)

    for name in val_fileNames:
        shutil.copy(name, ROOT_DATA_DIR + "/val/" + current_class)

    for name in test_fileNames:
        shutil.copy(name, ROOT_DATA_DIR + "/test/" + current_class)

In [6]:
for cls in CLASSES:
    loadDataset(cls)

Burger 
	 Total images:  500
	 Training:  350
	 Validation:  75
	 Testing:  75
Dimsum 
	 Total images:  500
	 Training:  350
	 Validation:  75
	 Testing:  75
Ramen 
	 Total images:  500
	 Training:  350
	 Validation:  75
	 Testing:  75
Sushi 
	 Total images:  500
	 Training:  350
	 Validation:  75
	 Testing:  75


# Load data into Dataset object

In [7]:
import os

import numpy as np

from itertools import chain

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
BATCH_SIZE = 16
IMAGE_SIZE = 128
CLASS_MODE = 'categorical'

In [9]:
train_ds = image_dataset_from_directory(
    directory='dataset/train',
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE)
)
train_ds.batch(BATCH_SIZE)

##########

validation_ds = image_dataset_from_directory(
    directory='dataset/val',
    labels='inferred',
    label_mode='categorical',
    shuffle=False,
    image_size=(IMAGE_SIZE, IMAGE_SIZE))
validation_ds.batch(BATCH_SIZE)

##########

test_ds = image_dataset_from_directory(
    directory='dataset/test',
    labels='inferred',
    label_mode='categorical',
    shuffle=False,
    image_size=(IMAGE_SIZE, IMAGE_SIZE))
test_ds.batch(BATCH_SIZE)


Found 1400 files belonging to 4 classes.
Found 300 files belonging to 4 classes.
Found 300 files belonging to 4 classes.


<BatchDataset element_spec=(TensorSpec(shape=(None, None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 4), dtype=tf.float32, name=None))>

# Create the model

### Model

In [27]:
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Resizing, Rescaling, RandomFlip, RandomRotation, Conv2D, AveragePooling2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Activation, ReLU
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array 
from tensorflow.keras.models import load_model

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

### Custom

In [11]:
model = keras.Sequential(
    [
    Input(shape = (IMAGE_SIZE, IMAGE_SIZE , 3), ),
    Resizing(IMAGE_SIZE,IMAGE_SIZE),
    Rescaling(scale=1./255),
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(factor=(0.5,0.5)),
     
    Conv2D(128, (1,1), strides = 1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),

    Conv2D(64, (1,1), strides=1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    Conv2D(64, (3,3), strides=2),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    Conv2D(128, (1,1), strides=1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    MaxPooling2D(pool_size=(2,2)),
              
    Conv2D(128, (1,1), strides=1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    Conv2D(128, (3,3), strides=2),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    Conv2D(256, (1,1), strides=1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(256, (1,1), strides=1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    Conv2D(512, (3,3), strides=2),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),
    Conv2D(512, (1,1), strides=1),
    BatchNormalization(),
    Activation(activation = "leaky_relu"),

    Conv2D(64, (3,3), strides=1),
    Activation(activation = "leaky_relu"),
    MaxPooling2D(pool_size=(2,2)),

    AveragePooling2D(1),
    Flatten(),
    Dense(256),
    Activation(activation = "leaky_relu"),
    Dense(128),
    Activation(activation = "leaky_relu"),
    Dense(64),
    Activation(activation = "leaky_relu"),
    Dense(32),
    Activation(activation = "leaky_relu"),
    Dense(4),

    Activation('softmax')
    ]
)

ValueError: Exception encountered when calling layer "max_pooling2d_2" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_2/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,64].

Call arguments received by layer "max_pooling2d_2" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 1, 1, 64), dtype=float32)

### ResNet50V2

In [28]:
import tensorflow as tf

model = tf.keras.applications.resnet_v2.ResNet50V2(
  weights='imagenet',
  include_top=False,
  input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
)
###
xc1 = Conv2D(64, (3,3), padding="same")(model.output)
xc2 = BatchNormalization()(xc1)
xc3 = ReLU()(xc2)

xmp1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(xc3)

f1 = Flatten()(xc3)

d1 = Dense(1024, activation='leaky_relu')(f1)
d2 = Dense(512, activation='leaky_relu')(d1)
d3 = Dense(128, activation='leaky_relu')(d2)
d4 = Dense(16, activation='leaky_relu')(d3)
output = Dense(4, activation='softmax')(d4)
###

model = Model(model.input, output)

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']         

In [16]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer, 
    loss='categorical_crossentropy', 
    metrics=['accuracy' , precision_m, recall_m , f1_m])

# Train the model

## Callbacks

Checkpoint

In [31]:
MODEL_NAME = 'resnet50v2.h5'

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    MODEL_NAME,
    verbose=1,
    monitor='val_accuracy',
    mode='max',
    save_best_only = True)

Early Stopping

In [32]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=20)

## Train

In [33]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [34]:
h = model.fit(
    train_ds, 
    epochs=100, 
    batch_size=BATCH_SIZE, 
    callbacks=[early_stopping, checkpoint], 
    validation_data=validation_ds)

Epoch 1/100
44/44 [==============================] - ETA: 0s - loss: 0.9766 - accuracy: 0.6029 - precision_m: 0.7630 - recall_m: 0.4441 - f1_m: 0.5415
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to resnet50v2.h5
44/44 [==============================] - 180s 3s/step - loss: 0.9766 - accuracy: 0.6029 - precision_m: 0.7630 - recall_m: 0.4441 - f1_m: 0.5415 - val_loss: 269.3482 - val_accuracy: 0.2500 - val_precision_m: 0.2344 - val_recall_m: 0.2344 - val_f1_m: 0.2344
Epoch 2/100
44/44 [==============================] - ETA: 0s - loss: 0.7471 - accuracy: 0.7379 - precision_m: 0.8112 - recall_m: 0.6087 - f1_m: 0.6865
Epoch 2: val_accuracy improved from 0.25000 to 0.43667, saving model to resnet50v2.h5
44/44 [==============================] - 185s 4s/step - loss: 0.7471 - accuracy: 0.7379 - precision_m: 0.8112 - recall_m: 0.6087 - f1_m: 0.6865 - val_loss: 16.4773 - val_accuracy: 0.4367 - val_precision_m: 0.4103 - val_recall_m: 0.4094 - val_f1_m: 0.4098
Epoch 3/100
44/44 

KeyboardInterrupt: 

In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.legend(['train', 'val'])